The below was done as an exercise from the "Computational Methods for Linear Algebra" course (MT404) with Alberto Saa.

Exercise proposition, in Brazillian Portuguese, is [EP3](http://vigo.ime.unicamp.br/mt404/EP3.pdf).

In short, the goal is to explore a subcubic algorithm for matrix multiplication, i.e. an algo with O(n^k) operations (k<3).

In [1]:
import numpy as np  # using for better initialization of arrays
import time

In [2]:
def vanilla_product(A: np.ndarray, B: np.ndarray) -> tuple[np.ndarray, float]:
    m, p = A.shape
    q, n = B.shape
    if p != q:
        raise ValueError(f"Dimension of A and B do not match")
    C = np.zeros((m, n))
    start = time.process_time()
    for i in range(m):
        for j in range(n):
            for k in range(p):
                C[i, j] += A[i, k] * B[k, j]
    end = time.process_time()
    return C, end - start

In [3]:
def winograd_product(A: np.ndarray, B: np.ndarray) -> tuple[np.ndarray, float]:
    m, p = A.shape
    q, n = B.shape
    if m != p or q != n or m != n or (n % 2 != 0):
        raise ValueError(f"Dimension of A and B have to be the same and even")
    C = np.zeros((n, n))
    v = np.zeros(n)
    w = np.zeros(n)
    start = time.process_time()
    for i in range(n):
        for k in range(n // 2):
            v[i] += A[i, 2 * k] * A[i, 2 * k + 1]
        for j in range(n):
            if w[j] == 0:  # only calculate when such entry was not already done
                for k in range(n // 2):    
                    w[j] += B[2 * k, j] * B[2 * k + 1, j]
            for k in range(n // 2):
                c = (A[i, 2 * k] + B[2 * k + 1, j]) * (A[i, 2 * k + 1] + B[2 * k, j])
                C[i, j] += c
            C[i, j] -= v[i]
            C[i, j] -= w[j]
    end = time.process_time()
    return C, end - start

In [4]:
n = 100
A = np.random.rand(n, n)
B = np.random.rand(n, n)
C_vanilla, time_vanilla = vanilla_product(A, B)
C_winograd, time_winograd = winograd_product(A, B)
# check if the product is correct
C = A @ B
epsilon = 1e-8
print(f"{n=}")
print(f"The vanilla product is correct: {np.linalg.norm(C-C_vanilla)<epsilon}")
print(f"The Winograd product is correct: {np.linalg.norm(C-C_winograd)<epsilon}")
print(f"Time for vanilla product = {time_vanilla:.2f}s")
print(f"Time for Winograd product = {time_winograd:.2f}s")

n=100
The vanilla product is correct: True
The Winograd product is correct: True
Time for vanilla product = 0.59s
Time for Winograd product = 0.54s


The ratio of floating point opertions in the Winograd versus the usual/standard (henceforth called "vanilla") product is (for $n\rightarrow\infty$)
$$r = \frac{3+\omega}{2+2\omega}$$
And thus it is needed that $\omega > 1$ for Winograd's algorithm to be more efficient than the vanilla one.
$$\omega = \frac{3 - 2r}{2r - 1}$$
We can proxy $r$ using the time it takes for the Winograd's over the time it takes for the vanilla product to run.

Using our sophisticated knowledge of statistics we will perform a very complex task for the estimation of the average times (we will use sample mean and variance, with sample size $M$).
$$\bar{T}=\frac{1}{M}\sum_{k=1}^M T_k \ \land \ S^2=\frac{1}{M-1}\sum_{k=1}^M (T_k-\bar{T})^2$$

For the average ratio, the mean will simply be
$$\bar{r} = \frac{\bar{T}_W}{\bar{T}_v}$$
Where $T_W$ is Winograd's product time and $T_v$ is the vanilla product time.

But, for the standard error, we will need to invoke an even more sophisticated techinque called "propagation of uncertainty" directly from the physics lab class. Then
$$\hat{\sigma}_r = \frac{1}{\sqrt{M}} \sqrt{ S^2_W\frac{1}{\bar{T}_v^2} + S^2_v\frac{\bar{T}_W^2}{\bar{T}_v^4}}=\frac{1}{\bar{T}_v\sqrt{M}}\sqrt{ S^2_W + S^2_v\frac{\bar{T}_W^2}{\bar{T}_v^2}}$$

Finally, doing the same for $\omega$, which is what we are *actually* interested in
$$\hat{\omega} = \frac{3 - 2\hat{r}}{2\hat{r} - 1} \ \land \ \hat{\sigma}_{\omega}=\frac{4\hat{\sigma}_r}{(2\hat{r}-1)^2}$$

In [5]:
def test(sample_size: int, n=100) -> None:
    M = sample_size
    vanilla = []
    wino = []
    for _ in range(M):
        A = np.random.rand(n, n)
        B = np.random.rand(n, n)
        C_vanilla, time_vanilla = vanilla_product(A, B)
        C_winograd, time_winograd = winograd_product(A, B)
        vanilla.append(time_vanilla)
        wino.append(time_winograd)
    vanilla_mean = np.mean(vanilla)
    vanilla_std = np.std(vanilla, ddof=1)
    wino_mean = np.mean(wino)
    wino_std = np.std(vanilla, ddof=1)
    r = wino_mean/vanilla_mean
    sigma = np.sqrt(wino_std**2 + vanilla_std**2 * (wino_mean/vanilla_mean)**2)/(vanilla_mean*np.sqrt(M))
    omega = (3 - 2*r)/(2*r - 1)
    error = 4 * sigma/(2*r - 1)**2
    print(f"Dimension: {n=}")
    print(f"Sample size: {M=}")
    print(f"Time for vanilla: {vanilla_mean:.2f} \pm {vanilla_std/np.sqrt(M):.2f}")
    print(f"Time for Winograd: {wino_mean:.2f} \pm {wino_std/np.sqrt(M):.2f}")
    print(f"Estimated {omega=:.2f} with {error=:.2f}")

In [6]:
test(10, n=100)

Dimension: n=100
Sample size: M=10
Time for vanilla: 0.62 \pm 0.01
Time for Winograd: 0.61 \pm 0.01
Estimated omega=1.05 with error=0.12


In [7]:
test(10, n=200)

Dimension: n=200
Sample size: M=10
Time for vanilla: 5.17 \pm 0.08
Time for Winograd: 5.27 \pm 0.08
Estimated omega=0.93 with error=0.09


In [8]:
test(10, n=300)

Dimension: n=300
Sample size: M=10
Time for vanilla: 18.88 \pm 0.22
Time for Winograd: 17.59 \pm 0.22
Estimated omega=1.32 with error=0.09


In [9]:
test(10, n=50)

Dimension: n=50
Sample size: M=10
Time for vanilla: 0.09 \pm 0.00
Time for Winograd: 0.08 \pm 0.00
Estimated omega=1.06 with error=0.17


In [10]:
test(100, n=100)

Dimension: n=100
Sample size: M=100
Time for vanilla: 0.72 \pm 0.01
Time for Winograd: 0.71 \pm 0.01
Estimated omega=1.07 with error=0.06


In [11]:
test(100, n=200)

Dimension: n=200
Sample size: M=100
Time for vanilla: 5.98 \pm 0.06
Time for Winograd: 5.86 \pm 0.06
Estimated omega=1.08 with error=0.07


In [12]:
test(100, n=300)

Dimension: n=300
Sample size: M=100
Time for vanilla: 16.07 \pm 0.17
Time for Winograd: 15.38 \pm 0.17
Estimated omega=1.19 with error=0.07


In [13]:
test(100, n=50)

Dimension: n=50
Sample size: M=100
Time for vanilla: 0.07 \pm 0.00
Time for Winograd: 0.07 \pm 0.00
Estimated omega=0.98 with error=0.04


The result is not exactly convincing, although (mostly) positive for Winorgrad's algorithm.

The superiority over the vanilla matrix multiplication algorithm is mild, at best, from this implementation and test.

This could be due to inefficiencies in the implementation, sure, but it is also possible that the processor has memory accessing/registering instructions that are not optimized for an "unusual" procedure for matrix multiplication. Meaning that it is possible that the difference would be more prominent in case the processor was completely agnostic to matrix products, given memory access is a cost not included in the calculations for the complexity.

Of course, there is also a limitation in such claim, for the language used in the above prototype is far from low level, so it is also possible that the results could change (for better or worse) in a low level language like C, or Fortran, or even lower, Assembly.

Is is *worth* using Winograd's algorithm, at least in python applications given the slight benefit seen above? I don't think so, mainly because NumPy's matrix multiplication is much faster and, if the matrix size is big enough for the time shaved to matter, it is very likely that space complexity will also come into play, so it is not obvious that Winograd's algorithm should be used (or even when it should be used).

In [14]:
n = 100
A = np.random.rand(n, n)
B = np.random.rand(n, n)
start = time.process_time()
C = A @ B
end = time.process_time()
time_elapsed = end - start
print(f"Time for NumPy's product = {time_elapsed:.2f}s")

Time for NumPy's product = 0.01s
